# Plot TPC-H results

This currently assumes that benchmarks have been run and have populated benchmark.db.  It also assumes that that database has only those results and from only one run (this is usually a bad assumption).

```
rm benchmark.db
pytest --benchmark tests/benchmarks/tpch --tpch-non-dask
```

In [ ]:
import ibis
ibis.options.interactive = True
con = ibis.sqlite.connect("benchmark.db")
t = con.table("test_run")
t

In [ ]:
tt = t[(t.call_outcome == "passed") & (t.path.startswith("tpch/"))]
tt[["path", "name", "duration", "start"]]

In [ ]:
df = tt[["path", "name", "duration", "start"]].to_pandas()

df["library"] = df.path.map(lambda path: path.split("_")[-1].split(".")[0])
df["query"] = df.name.map(lambda name: int(name.split("_")[-1]))
del df["path"]
del df["name"]
df = df.sort_values(["query", "library"])

def recent(df):
    return df.sort_values("start").iloc[-1]

df = df.groupby(["library", "query"]).apply(recent).reset_index(drop=True)
del df["start"]
df.head(10)

In [ ]:
import altair as alt

alt.Chart(df).mark_bar().encode(
    x="query:N",
    y="duration:Q",
    xOffset="library:N",
    color="library:N",
    tooltip=["library", "duration"],
)